In [1]:
import os
script_name = 'Test_20180928.ipynb'
dir_path = os.path.dirname(os.path.realpath(script_name))
parent_path = os.path.dirname(dir_path)
Func_path = os.path.join(parent_path,'Func')

import sys
sys.path.append(Func_path)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from loading_data import DataLoader_base


In [3]:
dataset = DataLoader_base(data_source="inside",filename="mnist")

/home/weixin/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/home/weixin/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/weixin/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
print(dataset.)